In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import CrostonOptimized, ADIDA, TSB

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
plt.rcParams["figure.figsize"] = (9,6)

In [ ]:
def errors(y_true, y_pred):
    
    # CFE
    cfe_all = np.cumsum(y_true - y_pred)
    cfe = cfe_all.iloc[-1]
    
    cfe_max = np.max(cfe_all)
    cfe_min = np.min(cfe_all)
    
    # PIS
    pis_all = -np.cumsum(cfe_all)
    pis = pis_all.iloc[-1]
    
    # NOS
    nos = len(cfe_all[cfe_all > 0])

    errors = {
        "CFE": cfe,
        "CFE_max": cfe_max,
        "CFE_min": cfe_min,
        "PIS": pis,
        "NOS": nos
    }
    
    return errors

In [ ]:
df = pd.read_csv('data/car_parts_monthly_sales.csv')

df['date'] = pd.to_datetime(df['date'])

df.head()

In [ ]:
# list unique parts_id


In [ ]:
parts_ids = df['parts_id'].unique()

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    
    ax.bar(
        x=df[df['parts_id'] == parts_ids[i]]['date'],
        height=df[df['parts_id'] == parts_ids[i]]['volume'],
        color='lightgray'
    )

    ax.set_xlabel('Date')
    ax.set_ylabel('Volume')
    
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    
plt.tight_layout()

## Forecasting 

In [ ]:
# Format dataset:
# Drop the id column (not the parts_id column)
# Rename columns to "unique_id", "ds", "y"



In [ ]:
# Use a horizon of 4 months
# Use a step size of 4
# Use 3 windows
# Model with Croston, ADIDA and TSB



## Evaluation 

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    
    ax.bar(
        x=df[df['parts_id'] == parts_ids[i]]['date'],
        height=df[df['parts_id'] == parts_ids[i]]['volume'],
        color='lightgray'
    )
    ax.plot(
        cv_df[cv_df.index==parts_ids[i]]['ds'],
        cv_df[cv_df.index==parts_ids[i]]['CrostonOptimized'],
        ls='--',
        label='Croston'
    )
    
    ax.plot(
        cv_df[cv_df.index==parts_ids[i]]['ds'],
        cv_df[cv_df.index==parts_ids[i]]['ADIDA'],
        ls=':',
        label='ADIDA'
    )
    
    ax.plot(
        cv_df[cv_df.index==parts_ids[i]]['ds'],
        cv_df[cv_df.index==parts_ids[i]]['TSB'],
        ls='-.',
        label='TSB'
    )

    ax.set_xlabel('Date')
    ax.set_ylabel('Volume')
    ax.legend(loc='best')
    
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    
plt.tight_layout()

In [ ]:
croston_errors = errors(cv_df['y'], cv_df['CrostonOptimized'])
adida_errors = errors(cv_df['y'], cv_df['ADIDA'])
tsb_errors = errors(cv_df['y'], cv_df['TSB'])

print(croston_errors)
print(adida_errors)
print(tsb_errors)

In [ ]:
models = ['Croston', 'ADIDA', 'TSB']
errors = {
    'CFE': (-41, -5, -34),
    'PIS': (2060, 270, 1424),
    'NOS': (12, 26, 14)
}

x = np.arange(len(models))
width = 0.25
multiplier = 0

fig, ax = plt.subplots()

for attr, value in errors.items():
    offset = width*multiplier
    rects = ax.bar(x+offset, value, width, label=attr)
    ax.bar_label(rects, padding=3)
    multiplier += 1
    
ax.set_xlabel('Models')
ax.set_ylabel('Error metrics')
ax.set_xticks(x+width, models)
ax.legend(loc='best')

plt.tight_layout()